In [18]:
import warnings
warnings.filterwarnings('ignore')

from utils import *
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.impute import KNNImputer, MissingIndicator
from sklearn.feature_selection import mutual_info_classif, f_classif

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats

In [2]:
df = pd.read_csv("kaggle_data/predict_sex_train_utf.csv", index_col = 'Id')
df_test = pd.read_csv("kaggle_data/predict_sex_test_no_answers_no_mv.csv", index_col = 'Id')

In [3]:
cols = ['Sex',
    'НЕВЕРОТН 1  Невероятные приключения итальянцев в России',
 'ТЕЛОХРАН 2  Телохранитель',
 'КАВКАЗСК 3  Кавказская пленница или Новые приключения Шурика',
 'ПТЫЙ_ЭЛЕ 4  Пятый элемент',
 'ТЕРМИНАТ 5  Терминатор',
 'КРЕПКИЙ 6  Крепкий орешек',
 'ПОЛИЦ_АК 7  Полицейская академия',
 'ЧУЖИЕ 8  Чужие',
 'КРОКОДИЛ 9  Крокодил Данди',
 'ЗДРАВСТВ 10  Здравствуйте  я ваша тетя!',
 'ИРОНИ_СУ 11  Ирония судьбы или с легким паром',
 'КРАСОТКА 12  Красотка',
 'МАТРИЦА 13  Матрица',
 'ТИТАНИК 14  Титаник',
 'К_9 15  К-9',
 'НАЗАД_В 16  Назад в будущее',
 'САМА_ОБА 17  Сама обательная и привлекательна',
 'БЛИЗНЕЦЫ 18  Близнецы',
 'СОБАЧЬЕ 19  Собачье сердце',
 'СОБАКА_Б 20  Собака Баскервилей',
 'МЕСТО_ВС 21  Место встречи изменить нельзя',
 'ГОРЕЦ 22  Горец',
 'СЛУЖЕБНЫ 23  Служебный роман',
 'ОС_РЫБ 25  Особенности национальной рыбалки',
 'ОС_ОХОТЫ 26  Особенности национальной охоты',
 'ДЕСТЬ_НЕ 27  Десть негритят',
 'ТЕРМИНA2 28  Терминатор 2: судный день',
 'ВОКЗАЛ_Д 29  Вокзал для двоих',
 'ГЕНИЙ 30  Гений',
 'ИВАН_ВАС 31  Иван Васильевич меняет профессию',
 'РОККИ 32  Рокки',
 'ОДИН_ДОМ 33  Один дома',
 'САМОГОНЩ 34  Самогонщики',
 'ОБЫКНОВЕ 35  Обыкновенное чудо',
 'В_БОЙ_ИД 36  В бой идут одни старики',
 'ПАРК_ЮРС 37  Парк Юрского периода',
 'МОСКВА_С 38  Москва слезам не верит',
 'БЕЛОЕ_СО 39  Белое солнце пустыни',
 'ОДИН_ДМ2 41  Один дома 2: потерянный в Нью-Йорке',
 'ХИЩНИК 42  Хищник',
 'ВСПОМНИТ 43  Вспомнить все',
 'ИНТЕРДЕВ 44  Интердевочка',
 'ЧЕЛЮСТИ 45  Челюсти',
 'ЛЕОН 46  Леон',
 'ПОЛИЦ_ДС 47  Полицейский из детского сада',
 'ОТ_ЗАКАТ 48  От заката до рассвета',
 'АРМАГЕДД 50  Армагеддон',
 'ДВОЙНОЙ 52  Двойной удар',
 'МАСКА 53  Маска',
 'БЛЕФ 54  Блеф',
 'ДЕНЬ_СУР 56  День сурка',
 'БЕГУЩИЙ 57  Бегущий человек',
 'ТУТСИ 59  Тутси',
 'НА_ДЕРИБ 60  На Дерибасовской хороша погода или на Брайтон Бич всегда идут дожди',
 'ЧАРОДЕИ 62  Чародеи',
 'ЗУБАСТИК 64  Зубастики',
 'ИГРУШКА 65  Игрушка (Франция)',
 'ВЫСОКИЙ 66  Высокий блондин в черном ботинке',
 'КРЕПК_2 71  Крепкий орешек 2',
 'МОЛЧАНИЕ 73  Молчание ягнят',
 'ЭКИПАЖ 74  Экипаж',
 'УКРОЩЕНИ 78  Укрощение строптивого',
 'ПРОФЕССИ 79  Профессионал',
 'В_ДЖАЗЕ 85  В джазе только девушки',
        
 'E_I__КЕЙ 104 E/I (тест Кейpси)',
 'S_N__КЕЙ 105 S/N (тест Кейpси)',
 'T_F__КЕЙ 106 T/F (тест Кейpси)',
 'J_P__КЕЙ 107 J/P (тест Кейpси)',
 'ВОЗPАСТ 139 Возpаст',
 'ОБPАЗОВА 140 Обpазование',
 'УPОВЕНЬ 141 Уpовень доходов']

In [4]:
df = df[cols]
df_test = df_test[cols[1:]]
df.shape, df_test.shape

((1000, 72), (1859, 71))

In [5]:
# nan_columns_train = []
# for column in df.columns[1:]:
#     nan_ratio = np.isnan(df[column]).sum() / (len(df) - 1)
#     if nan_ratio > 0.65:
#         nan_columns_train.append(column)

# nan_columns_test = []
# for column in df_test.columns:
#     nan_ratio = np.isnan(df_test[column]).sum() / len(df_test)
#     if nan_ratio > 0.65:
#         nan_columns_test.append(column)
# for_drops = list(set(nan_columns_train + nan_columns_test))

In [6]:
# df = df.drop(for_drops, axis = 1)
# df_test = df_test.drop(for_drops, axis = 1)

# df.shape, df_test.shape

In [7]:
df.shape[1]

72

In [20]:
barrier = df.shape[1] - 1 - 7

prep = Preprocessor(1, 0)
prep_df = prep.fit_transform(df, 'Sex')
X, y = prep_df.drop('Sex', axis = 1), prep_df.Sex
y = y.astype(np.int64)
X.iloc[:, :barrier] = np.round(X.iloc[:, :barrier])
X.iloc[:, :barrier][X.iloc[:, :barrier] < 1] = 1
X.iloc[:, :barrier][X.iloc[:, :barrier] > 12] = 12

# X = prep.fill_gaps(X)
# # imputer = KNNImputer(n_neighbors=5, weights = 'distance')
# # X = imputer.fit_transform(X)
# X[:, :100] = np.round(X[:, :100])
film_means = np.round(X.iloc[:, :barrier].mean())
test_means = X.iloc[:, barrier:].mean()
X.iloc[:, :barrier] = X.iloc[:, :barrier].fillna(value = film_means)
X.iloc[:, barrier:] = X.iloc[:, barrier:].fillna(value = test_means)

len means: 71


In [9]:
(X.shape, y.shape), df.shape

(((890, 71), (890,)), (1000, 72))

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# GradientBoosting GridSearch

In [548]:
params = {
    # "loss":["deviance"],
    # "learning_rate": [0.01, 0.1],
    "min_samples_split": np.linspace(0.1, 0.5, 6),
    "min_samples_leaf": np.linspace(0.1, 0.5, 6),
    "max_depth": [1,2,3, 5, 8, 11, 14, 18, 22],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "squared_error"],
    "subsample":[0.5, 0.65, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[50, 100, 150, 200, 250]
    }

In [549]:
clf_gb = RandomizedSearchCV(GradientBoostingClassifier(random_state = 42), params, cv=10, n_jobs=-1, verbose=1, n_iter = 1000)
clf_gb.fit(X, y)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


RandomizedSearchCV(cv=10, estimator=GradientBoostingClassifier(random_state=42),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'criterion': ['friedman_mse',
                                                      'squared_error'],
                                        'max_depth': [1, 2, 3, 5, 8, 11, 14, 18,
                                                      22],
                                        'max_features': ['log2', 'sqrt'],
                                        'min_samples_leaf': array([0.1 , 0.18, 0.26, 0.34, 0.42, 0.5 ]),
                                        'min_samples_split': array([0.1 , 0.18, 0.26, 0.34, 0.42, 0.5 ]),
                                        'n_estimators': [50, 100, 150, 200,
                                                         250],
                                        'subsample': [0.5, 0.65, 0.8, 0.85, 0.9,
                                                      0.95, 1.0]},
                   ver

In [550]:
print(clf_gb.best_params_)
print(clf_gb.best_score_)
print(clf_gb.best_estimator_)

{'subsample': 0.65, 'n_estimators': 150, 'min_samples_split': 0.5, 'min_samples_leaf': 0.1, 'max_features': 'sqrt', 'max_depth': 2, 'criterion': 'squared_error'}
0.7752808988764045
GradientBoostingClassifier(criterion='squared_error', max_depth=2,
                           max_features='sqrt', min_samples_leaf=0.1,
                           min_samples_split=0.5, n_estimators=150,
                           random_state=42, subsample=0.65)


In [11]:
kwargs = {'subsample': 0.65, 'n_estimators': 150, 'min_samples_split': 0.5, 'min_samples_leaf': 0.1, 'max_features': 'sqrt', 'max_depth': 2, 'criterion': 'squared_error'}

# Predict

In [21]:
df_test = pd.read_csv("kaggle_data/predict_sex_test_no_answers_no_mv.csv", index_col = 'Id')
df_test = df_test[cols[1:]]
# df_test = prep.transform(df_test, bTrain=False)
# df_test= prep.fill_gaps(df_test)
df_test_index = df_test.index
print(df_test.iloc[:, barrier:].columns)
df_test.iloc[:, :barrier] = np.round(df_test.iloc[:, :barrier])
df_test.iloc[:, :barrier][df_test.iloc[:, :barrier] < 1] = 1
df_test.iloc[:, :barrier][df_test.iloc[:, :barrier] > 12] = 12
# nan_mask = indicator.transform(df_test)
# df_test = imputer.transform(df_test)
# df_test[nan_mask] = np.round(df_test[nan_mask])
df_test.iloc[:, :barrier] = df_test.iloc[:, :barrier].fillna(value = film_means)
df_test.iloc[:, barrier:] = df_test.iloc[:, barrier:].fillna(value = test_means)

#df_test = df_test[:, selected_cols]
df_test.shape

Index(['E_I__КЕЙ 104 E/I (тест Кейpси)', 'S_N__КЕЙ 105 S/N (тест Кейpси)',
       'T_F__КЕЙ 106 T/F (тест Кейpси)', 'J_P__КЕЙ 107 J/P (тест Кейpси)',
       'ВОЗPАСТ 139 Возpаст', 'ОБPАЗОВА 140 Обpазование',
       'УPОВЕНЬ 141 Уpовень доходов'],
      dtype='object')


(1859, 71)

In [ ]:
answers_array = clf_gb.best_estimator_.predict(df_test)
answers_array = answers_array.astype(np.int64)

In [595]:
answers_array

array([2, 2, 2, ..., 1, 1, 1])

In [596]:
answers_array

array([2, 2, 2, ..., 1, 1, 1])

In [597]:
answers = pd.DataFrame(answers_array, columns = ['Sex'], index = df_test_index)
answers.to_csv('answers.csv')

In [35]:
asvm = clf_svm.best_estimator_.predict(df_test)
arf = clf_rf.best_estimator_.predict(df_test)
agb = clf_gb.best_estimator_.predict(df_test)

In [42]:
atriple = np.round((asvm + arf + agb) / 3)

In [44]:
(atriple != asvm).sum(), (atriple != arf).sum(), (atriple != agb).sum()

(269, 185, 85)

In [49]:
atriple = atriple.astype(np.int64)
atriple

array([2, 2, 2, ..., 1, 1, 2])

In [50]:
answers = pd.DataFrame(atriple, columns = ['Sex'], index = df_test_index)
answers.to_csv('answers.csv')